<a href="https://colab.research.google.com/github/MinJunJA/babymodel/blob/master/crnn_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Reshape
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow as tf

# 데이터셋 디렉토리 경로 설정
data_dir = '/content/drive/MyDrive/data2'

# 클래스 레이블 설정 (클래스별 디렉토리 이름으로 가정)
class_labels = os.listdir(data_dir)

# MFCC 계수 개수 설정
n_mfcc = 12

# 데이터셋과 레이블을 저장할 리스트
X = []
labels = []

# 데이터셋 불러오기
for label_idx, label in enumerate(class_labels):
    class_dir = os.path.join(data_dir, label)
    for filename in os.listdir(class_dir):
        file_path = os.path.join(class_dir, filename)
        y, sr = librosa.load(file_path, sr=16000)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        X.append(mfccs.T)
        labels.append(label)

# NumPy 배열로 변환
X = np.array(X)

# 레이블을 정수형으로 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
num_classes = len(class_labels)
labels = to_categorical(labels, num_classes=num_classes)

# 최대 길이에 맞춰 MFCC 벡터를 패딩
max_length = max(len(mfccs) for mfccs in X)
X = np.array([np.pad(mfccs, ((0, max_length - len(mfccs)), (0, 0)), mode='constant') for mfccs in X])

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
# 첫 번째 모델 생성
model1 = Sequential()
# 모델1의 레이어 추가 및 컴파일 설정

# Convolutional Layer 추가
model1.add(Conv2D(64, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
model1.add(MaxPooling2D((2, 2)))
model1.add(BatchNormalization())
model1.add(Dropout(0.3))  # Dropout 추가

# Convolutional Layer 추가
model1.add(Conv2D(128, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(BatchNormalization())
model1.add(Dropout(0.3))  # Dropout 추가

# Convolutional Layer 추가
model1.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2), padding='same'))
model1.add(BatchNormalization())
model1.add(Dropout(0.3))  # Dropout 추가

model1.add(Flatten())

# LSTM 레이어 변경
model1.add(Reshape((X_train.shape[1] // 8, (X_train.shape[2] // 8) * 256)))
model1.add(LSTM(128, return_sequences=True))
model1.add(Dropout(0.5))
model1.add(LSTM(64, return_sequences=True))
model1.add(LSTM(32))
model1.add(Dropout(0.5))

# Output 레이어 변경
model1.add(Dense(num_classes, activation='softmax'))

# 모델1 컴파일
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 두 번째 모델 생성
model2 = Sequential()
# 모델2의 레이어 추가 및 컴파일 설정

# Convolutional Layer 추가
model2.add(Conv2D(64, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
model2.add(MaxPooling2D((2, 2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.3))  # Dropout 추가

# Convolutional Layer 추가
model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.3))  # Dropout 추가

# Convolutional Layer 추가
model2.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), padding='same'))
model2.add(BatchNormalization())
model2.add(Dropout(0.3))  # Dropout 추가

model2.add(Flatten())

# LSTM 레이어 변경
model2.add(Reshape((X_train.shape[1] // 8, (X_train.shape[2] // 8) * 256)))
model2.add(LSTM(128, return_sequences=True))
model2.add(Dropout(0.5))
model2.add(LSTM(64, return_sequences=True))
model2.add(LSTM(32))
model2.add(Dropout(0.5))

# Output 레이어 변경
model2.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
initial_lr = 0.001  # 초기 학습률 설정

# Learning Rate를 조정할 함수 정의
def lr_schedule(epoch):
    decay_rate = 0.9   # 학습률 감소 비율
    decay_steps = 10    # 학습률 감소 주기 (에폭 단위)
    lr = initial_lr * decay_rate**(epoch // decay_steps)
    return lr

# Learning Rate Scheduler 콜백 생성
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

# 모델 컴파일
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 두 모델 학습
model1.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.1, callbacks=[LearningRateScheduler(lr_schedule)])
model2.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.1, callbacks=[LearningRateScheduler(lr_schedule)])

# 각 모델의 예측 결과 가져오기
predictions1 = model1.predict(X_test)
predictions2 = model2.predict(X_test)

# 예측 결과 앙상블 (여기서는 간단히 평균 값을 사용)
ensemble_predictions = (predictions1 + predictions2) / 2

# 최종 예측 결과 평가
ensemble_accuracy = np.mean(np.argmax(ensemble_predictions, axis=1) == np.argmax(y_test, axis=1))
print("Ensemble Accuracy:", ensemble_accuracy)

Epoch 1/200
247/247 [==============================] - 24s 16ms/step - loss: 1.6321 - accuracy: 0.3494 - val_loss: 1.4881 - val_accuracy: 0.4075 - lr: 0.0010
Epoch 2/200
247/247 [==============================] - 3s 10ms/step - loss: 1.5012 - accuracy: 0.4196 - val_loss: 1.3659 - val_accuracy: 0.4646 - lr: 0.0010
Epoch 3/200
247/247 [==============================] - 2s 10ms/step - loss: 1.4203 - accuracy: 0.4595 - val_loss: 1.3244 - val_accuracy: 0.4989 - lr: 0.0010
Epoch 4/200
247/247 [==============================] - 3s 12ms/step - loss: 1.3540 - accuracy: 0.4948 - val_loss: 1.2295 - val_accuracy: 0.5445 - lr: 0.0010
Epoch 5/200
247/247 [==============================] - 3s 13ms/step - loss: 1.3000 - accuracy: 0.5185 - val_loss: 1.2074 - val_accuracy: 0.5594 - lr: 0.0010
Epoch 6/200
247/247 [==============================] - 2s 10ms/step - loss: 1.2473 - accuracy: 0.5404 - val_loss: 1.1684 - val_accuracy: 0.5559 - lr: 0.0010
Epoch 7/200
247/247 [==============================] - 2s